In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../saved_models/{model_name}'))

# Train on Real Test on Real

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [8]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.427680730819702, Final Batch Loss: 2.207463026046753
Epoch 2, Loss: 4.430686712265015, Final Batch Loss: 2.216057777404785
Epoch 3, Loss: 4.424543857574463, Final Batch Loss: 2.2134649753570557
Epoch 4, Loss: 4.425635814666748, Final Batch Loss: 2.216773509979248
Epoch 5, Loss: 4.419931888580322, Final Batch Loss: 2.206233501434326
Epoch 6, Loss: 4.418008327484131, Final Batch Loss: 2.210620403289795
Epoch 7, Loss: 4.418031692504883, Final Batch Loss: 2.214569091796875
Epoch 8, Loss: 4.413766145706177, Final Batch Loss: 2.2116448879241943
Epoch 9, Loss: 4.406684160232544, Final Batch Loss: 2.197998523712158
Epoch 10, Loss: 4.407388210296631, Final Batch Loss: 2.2048020362854004
Epoch 11, Loss: 4.402667284011841, Final Batch Loss: 2.1994669437408447
Epoch 12, Loss: 4.4007298946380615, Final Batch Loss: 2.201507568359375
Epoch 13, Loss: 4.397089719772339, Final Batch Loss: 2.20060133934021
Epoch 14, Loss: 4.384135961532593, Final Batch Loss: 2.175703287124634
Epoch 15, L

Epoch 134, Loss: 1.895640790462494, Final Batch Loss: 0.9014413952827454
Epoch 135, Loss: 1.9286581873893738, Final Batch Loss: 0.9512782692909241
Epoch 136, Loss: 1.7677077651023865, Final Batch Loss: 0.8514296412467957
Epoch 137, Loss: 1.8047423958778381, Final Batch Loss: 0.9092517495155334
Epoch 138, Loss: 1.8391600251197815, Final Batch Loss: 0.947961688041687
Epoch 139, Loss: 1.7866104245185852, Final Batch Loss: 0.8509378433227539
Epoch 140, Loss: 1.778935194015503, Final Batch Loss: 0.9150768518447876
Epoch 141, Loss: 1.817272126674652, Final Batch Loss: 0.942811906337738
Epoch 142, Loss: 1.891390323638916, Final Batch Loss: 0.8947718739509583
Epoch 143, Loss: 1.758235514163971, Final Batch Loss: 0.8392970561981201
Epoch 144, Loss: 1.6116593480110168, Final Batch Loss: 0.7692733407020569
Epoch 145, Loss: 1.6965954303741455, Final Batch Loss: 0.8963170647621155
Epoch 146, Loss: 1.7073152661323547, Final Batch Loss: 0.8989359736442566
Epoch 147, Loss: 1.7115889191627502, Final Ba

Epoch 259, Loss: 1.0779154300689697, Final Batch Loss: 0.5274035930633545
Epoch 260, Loss: 1.1625540256500244, Final Batch Loss: 0.6344157457351685
Epoch 261, Loss: 1.2747302651405334, Final Batch Loss: 0.6855672597885132
Epoch 262, Loss: 1.0580711662769318, Final Batch Loss: 0.4954887330532074
Epoch 263, Loss: 1.1215276718139648, Final Batch Loss: 0.4991387128829956
Epoch 264, Loss: 1.094895362854004, Final Batch Loss: 0.5618290901184082
Epoch 265, Loss: 1.1078624725341797, Final Batch Loss: 0.5289179682731628
Epoch 266, Loss: 1.15422785282135, Final Batch Loss: 0.6271396279335022
Epoch 267, Loss: 1.0685230493545532, Final Batch Loss: 0.5456312298774719
Epoch 268, Loss: 1.2121337056159973, Final Batch Loss: 0.610424816608429
Epoch 269, Loss: 1.1775811314582825, Final Batch Loss: 0.6590771675109863
Epoch 270, Loss: 1.1283026337623596, Final Batch Loss: 0.5956453084945679
Epoch 271, Loss: 1.147142231464386, Final Batch Loss: 0.5928599238395691
Epoch 272, Loss: 1.1747117638587952, Final 

Epoch 375, Loss: 0.9177896678447723, Final Batch Loss: 0.48070812225341797
Epoch 376, Loss: 1.0286922454833984, Final Batch Loss: 0.5566493272781372
Epoch 377, Loss: 1.0104429125785828, Final Batch Loss: 0.5500656962394714
Epoch 378, Loss: 0.847988098859787, Final Batch Loss: 0.42865613102912903
Epoch 379, Loss: 0.8794921040534973, Final Batch Loss: 0.4290086030960083
Epoch 380, Loss: 0.8538897931575775, Final Batch Loss: 0.38201695680618286
Epoch 381, Loss: 0.9061276912689209, Final Batch Loss: 0.4433367848396301
Epoch 382, Loss: 0.9601548612117767, Final Batch Loss: 0.4993897080421448
Epoch 383, Loss: 0.9535393714904785, Final Batch Loss: 0.4757559597492218
Epoch 384, Loss: 0.9478330910205841, Final Batch Loss: 0.40308573842048645
Epoch 385, Loss: 0.9417075216770172, Final Batch Loss: 0.4265904128551483
Epoch 386, Loss: 1.011982798576355, Final Batch Loss: 0.540845513343811
Epoch 387, Loss: 0.8460038006305695, Final Batch Loss: 0.4158148169517517
Epoch 388, Loss: 0.8444273173809052, 

Epoch 505, Loss: 0.7728246748447418, Final Batch Loss: 0.3576502501964569
Epoch 506, Loss: 0.8118916153907776, Final Batch Loss: 0.3717544674873352
Epoch 507, Loss: 0.7920491397380829, Final Batch Loss: 0.44200947880744934
Epoch 508, Loss: 0.8214981257915497, Final Batch Loss: 0.4409494698047638
Epoch 509, Loss: 0.892907589673996, Final Batch Loss: 0.48063206672668457
Epoch 510, Loss: 0.7852971255779266, Final Batch Loss: 0.3787388205528259
Epoch 511, Loss: 0.7882833182811737, Final Batch Loss: 0.4200178384780884
Epoch 512, Loss: 0.8218179047107697, Final Batch Loss: 0.4223887026309967
Epoch 513, Loss: 0.7829708755016327, Final Batch Loss: 0.39925435185432434
Epoch 514, Loss: 0.7803818583488464, Final Batch Loss: 0.3643655776977539
Epoch 515, Loss: 0.7633792161941528, Final Batch Loss: 0.39694079756736755
Epoch 516, Loss: 0.7839435338973999, Final Batch Loss: 0.3360777497291565
Epoch 517, Loss: 0.8199158012866974, Final Batch Loss: 0.43150442838668823
Epoch 518, Loss: 0.836047351360321

...

Epoch 4507, Loss: 0.19016344100236893, Final Batch Loss: 0.07743705809116364
Epoch 4508, Loss: 0.18810013681650162, Final Batch Loss: 0.10951566696166992
Epoch 4509, Loss: 0.25478334724903107, Final Batch Loss: 0.14881540834903717
Epoch 4510, Loss: 0.20884811133146286, Final Batch Loss: 0.06741594523191452
Epoch 4511, Loss: 0.15730737149715424, Final Batch Loss: 0.06339757144451141
Epoch 4512, Loss: 0.19327954947948456, Final Batch Loss: 0.09452477842569351
Epoch 4513, Loss: 0.24932564795017242, Final Batch Loss: 0.12047328054904938
Epoch 4514, Loss: 0.16863582283258438, Final Batch Loss: 0.08001817017793655
Epoch 4515, Loss: 0.15995769202709198, Final Batch Loss: 0.06170268356800079
Epoch 4516, Loss: 0.18986518681049347, Final Batch Loss: 0.08502933382987976
Epoch 4517, Loss: 0.2024524286389351, Final Batch Loss: 0.11873185634613037
Epoch 4518, Loss: 0.264144591987133, Final Batch Loss: 0.16214756667613983
Epoch 4519, Loss: 0.22337186336517334, Final Batch Loss: 0.09528052806854248
Ep

Epoch 4615, Loss: 0.1885722577571869, Final Batch Loss: 0.07525516301393509
Epoch 4616, Loss: 0.2546863630414009, Final Batch Loss: 0.09790585190057755
Epoch 4617, Loss: 0.20978309959173203, Final Batch Loss: 0.12162657827138901
Epoch 4618, Loss: 0.1472029611468315, Final Batch Loss: 0.057340264320373535
Epoch 4619, Loss: 0.2144128605723381, Final Batch Loss: 0.09031303972005844
Epoch 4620, Loss: 0.2478773146867752, Final Batch Loss: 0.1416628509759903
Epoch 4621, Loss: 0.1821102797985077, Final Batch Loss: 0.07649486511945724
Epoch 4622, Loss: 0.219694584608078, Final Batch Loss: 0.09567831456661224
Epoch 4623, Loss: 0.21575622260570526, Final Batch Loss: 0.09641988575458527
Epoch 4624, Loss: 0.14787491410970688, Final Batch Loss: 0.05477850139141083
Epoch 4625, Loss: 0.23710810393095016, Final Batch Loss: 0.11359991878271103
Epoch 4626, Loss: 0.17402151226997375, Final Batch Loss: 0.08088640868663788
Epoch 4627, Loss: 0.22652918100357056, Final Batch Loss: 0.1181633472442627
Epoch 46

Epoch 4741, Loss: 0.1716194450855255, Final Batch Loss: 0.07258529961109161
Epoch 4742, Loss: 0.17986281216144562, Final Batch Loss: 0.09452943503856659
Epoch 4743, Loss: 0.15536769852042198, Final Batch Loss: 0.05246401205658913
Epoch 4744, Loss: 0.1775304526090622, Final Batch Loss: 0.06338175386190414
Epoch 4745, Loss: 0.17728357017040253, Final Batch Loss: 0.07856166362762451
Epoch 4746, Loss: 0.15817942470312119, Final Batch Loss: 0.06928499788045883
Epoch 4747, Loss: 0.15654364973306656, Final Batch Loss: 0.060487352311611176
Epoch 4748, Loss: 0.22885124385356903, Final Batch Loss: 0.1228056252002716
Epoch 4749, Loss: 0.1781964935362339, Final Batch Loss: 0.05949438735842705
Epoch 4750, Loss: 0.2069481983780861, Final Batch Loss: 0.12814415991306305
Epoch 4751, Loss: 0.15839208662509918, Final Batch Loss: 0.06392897665500641
Epoch 4752, Loss: 0.2864968031644821, Final Batch Loss: 0.2164405882358551
Epoch 4753, Loss: 0.1478223130106926, Final Batch Loss: 0.06673088669776917
Epoch 

Epoch 4861, Loss: 0.20295438915491104, Final Batch Loss: 0.12180987745523453
Epoch 4862, Loss: 0.1704985424876213, Final Batch Loss: 0.09512745589017868
Epoch 4863, Loss: 0.1733236312866211, Final Batch Loss: 0.08537114411592484
Epoch 4864, Loss: 0.16138514131307602, Final Batch Loss: 0.08772565424442291
Epoch 4865, Loss: 0.19300898909568787, Final Batch Loss: 0.10036994516849518
Epoch 4866, Loss: 0.2289099395275116, Final Batch Loss: 0.11629708111286163
Epoch 4867, Loss: 0.15332218259572983, Final Batch Loss: 0.0408015102148056
Epoch 4868, Loss: 0.19450931996107101, Final Batch Loss: 0.09705938398838043
Epoch 4869, Loss: 0.11511939018964767, Final Batch Loss: 0.04928753525018692
Epoch 4870, Loss: 0.18257181346416473, Final Batch Loss: 0.1021391749382019
Epoch 4871, Loss: 0.29161337018013, Final Batch Loss: 0.1731395125389099
Epoch 4872, Loss: 0.17138046771287918, Final Batch Loss: 0.10139360278844833
Epoch 4873, Loss: 0.16688203811645508, Final Batch Loss: 0.0714954361319542
Epoch 487

Epoch 4985, Loss: 0.1648438200354576, Final Batch Loss: 0.08567185699939728
Epoch 4986, Loss: 0.16471297666430473, Final Batch Loss: 0.10533574223518372
Epoch 4987, Loss: 0.14333349838852882, Final Batch Loss: 0.05975921079516411
Epoch 4988, Loss: 0.1403031162917614, Final Batch Loss: 0.04058270528912544
Epoch 4989, Loss: 0.14046068117022514, Final Batch Loss: 0.05707496777176857
Epoch 4990, Loss: 0.20313109457492828, Final Batch Loss: 0.13895587623119354
Epoch 4991, Loss: 0.19033481925725937, Final Batch Loss: 0.0815543383359909
Epoch 4992, Loss: 0.1747642531991005, Final Batch Loss: 0.09596822410821915
Epoch 4993, Loss: 0.14298060908913612, Final Batch Loss: 0.08230508118867874
Epoch 4994, Loss: 0.2598283439874649, Final Batch Loss: 0.06997057795524597
Epoch 4995, Loss: 0.20758536458015442, Final Batch Loss: 0.08766692131757736
Epoch 4996, Loss: 0.1374574936926365, Final Batch Loss: 0.051866818219423294
Epoch 4997, Loss: 0.18277210742235184, Final Batch Loss: 0.09567105770111084
Epoc

In [51]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[22  0  0  0  0  0  0  0  0]
 [ 0  8  0  0  0  0  0  0  0]
 [ 0  0  7  0  0  1  0  0  0]
 [ 0  0  0  8  0  0  0  0  0]
 [ 0  0  0  0  8  0  0  0  0]
 [ 0  0  1  0  0  8  0  0  2]
 [ 0  0  0  0  0  0 12  0  0]
 [ 0  0  0  0  0  0  0 10  0]
 [ 0  0  0  0  0  1  1  0 11]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        22
           1    1.00000   1.00000   1.00000         8
           2    0.87500   0.87500   0.87500         8
           3    1.00000   1.00000   1.00000         8
           4    1.00000   1.00000   1.00000         8
           5    0.80000   0.72727   0.76190        11
           6    0.92308   1.00000   0.96000        12
           7    1.00000   1.00000   1.00000        10
           8    0.84615   0.84615   0.84615        13

    accuracy                        0.94000       100
   macro avg    0.93825   0.93871   0.93812       100
weighted avg    0.93877   0.94000   0.93901       100



# Train on Fake Test on Real

In [10]:
gen = Generator(z_dim = 106)
load_model(gen, "cGAN_UCI_30k_TEST_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=106, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=46, bias=True)
    (4): Tanh()
  )
)

In [24]:
latent_vectors = get_noise(len(X_train), 100)
act_vectors = get_act_matrix(len(X_train), 3)
usr_vectors = get_usr_matrix(len(X_train), 3)

fake_labels = []

for k in range(len(X_train)):
    if act_vectors[0][k] == 0 and usr_vectors[0][k] == 0:
        fake_labels.append(0)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 0:
        fake_labels.append(1)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 0:
        fake_labels.append(2)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 1:
        fake_labels.append(3)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 1:
        fake_labels.append(4)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 1:
        fake_labels.append(5)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 2:
        fake_labels.append(6)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 2:
        fake_labels.append(7)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 2:
        fake_labels.append(8)

fake_labels = np.asarray(fake_labels)
to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach().numpy()

In [25]:
model_fake = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fake.parameters(), lr = lr)

train_features = torch.tensor(fake_features)
train_labels = torch.tensor(fake_labels)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)

In [26]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_fake(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.437093257904053, Final Batch Loss: 2.2363758087158203
Epoch 2, Loss: 4.418625593185425, Final Batch Loss: 2.2023468017578125
Epoch 3, Loss: 4.4237470626831055, Final Batch Loss: 2.2248528003692627
Epoch 4, Loss: 4.41790509223938, Final Batch Loss: 2.2051563262939453
Epoch 5, Loss: 4.414108753204346, Final Batch Loss: 2.2095723152160645
Epoch 6, Loss: 4.411884546279907, Final Batch Loss: 2.208794116973877
Epoch 7, Loss: 4.406961917877197, Final Batch Loss: 2.2055656909942627
Epoch 8, Loss: 4.403103828430176, Final Batch Loss: 2.2030482292175293
Epoch 9, Loss: 4.400484323501587, Final Batch Loss: 2.195773124694824
Epoch 10, Loss: 4.395890712738037, Final Batch Loss: 2.1942098140716553
Epoch 11, Loss: 4.3940818309783936, Final Batch Loss: 2.1958820819854736
Epoch 12, Loss: 4.39016580581665, Final Batch Loss: 2.194636583328247
Epoch 13, Loss: 4.389580965042114, Final Batch Loss: 2.192624568939209
Epoch 14, Loss: 4.386880874633789, Final Batch Loss: 2.1937923431396484
Epoch

Epoch 129, Loss: 0.9406656324863434, Final Batch Loss: 0.47210320830345154
Epoch 130, Loss: 0.9619327783584595, Final Batch Loss: 0.46307358145713806
Epoch 131, Loss: 0.8034489452838898, Final Batch Loss: 0.39077162742614746
Epoch 132, Loss: 0.855337381362915, Final Batch Loss: 0.48018231987953186
Epoch 133, Loss: 0.9213784635066986, Final Batch Loss: 0.4747239053249359
Epoch 134, Loss: 0.8959013819694519, Final Batch Loss: 0.4161953628063202
Epoch 135, Loss: 0.9485061168670654, Final Batch Loss: 0.4677566587924957
Epoch 136, Loss: 0.8675930500030518, Final Batch Loss: 0.4345795512199402
Epoch 137, Loss: 0.8804813027381897, Final Batch Loss: 0.43033894896507263
Epoch 138, Loss: 0.8636233508586884, Final Batch Loss: 0.399203896522522
Epoch 139, Loss: 0.8703757524490356, Final Batch Loss: 0.4119870066642761
Epoch 140, Loss: 0.839065283536911, Final Batch Loss: 0.46316108107566833
Epoch 141, Loss: 0.8761338293552399, Final Batch Loss: 0.40917378664016724
Epoch 142, Loss: 0.888216555118560

Epoch 257, Loss: 0.5272809863090515, Final Batch Loss: 0.2007618546485901
Epoch 258, Loss: 0.5767721235752106, Final Batch Loss: 0.3136080205440521
Epoch 259, Loss: 0.49096377193927765, Final Batch Loss: 0.25180861353874207
Epoch 260, Loss: 0.43819141387939453, Final Batch Loss: 0.17485252022743225
Epoch 261, Loss: 0.5044281482696533, Final Batch Loss: 0.3054131865501404
Epoch 262, Loss: 0.5229319036006927, Final Batch Loss: 0.25037261843681335
Epoch 263, Loss: 0.5012392699718475, Final Batch Loss: 0.23495802283287048
Epoch 264, Loss: 0.5028478801250458, Final Batch Loss: 0.2439962923526764
Epoch 265, Loss: 0.5042735487222672, Final Batch Loss: 0.21693600714206696
Epoch 266, Loss: 0.5502733290195465, Final Batch Loss: 0.2817154824733734
Epoch 267, Loss: 0.42216956615448, Final Batch Loss: 0.2214755415916443
Epoch 268, Loss: 0.5340834558010101, Final Batch Loss: 0.28564921021461487
Epoch 269, Loss: 0.5357418209314346, Final Batch Loss: 0.22642509639263153
Epoch 270, Loss: 0.573041737079

Epoch 380, Loss: 0.32109619677066803, Final Batch Loss: 0.13176365196704865
Epoch 381, Loss: 0.45743606984615326, Final Batch Loss: 0.27321016788482666
Epoch 382, Loss: 0.3984605520963669, Final Batch Loss: 0.20301203429698944
Epoch 383, Loss: 0.403827428817749, Final Batch Loss: 0.21497811377048492
Epoch 384, Loss: 0.39774374663829803, Final Batch Loss: 0.17108739912509918
Epoch 385, Loss: 0.2928582429885864, Final Batch Loss: 0.1566718965768814
Epoch 386, Loss: 0.33111472427845, Final Batch Loss: 0.1552625447511673
Epoch 387, Loss: 0.4307364672422409, Final Batch Loss: 0.23544612526893616
Epoch 388, Loss: 0.34092067182064056, Final Batch Loss: 0.11965347826480865
Epoch 389, Loss: 0.40848036110401154, Final Batch Loss: 0.18975386023521423
Epoch 390, Loss: 0.41464921832084656, Final Batch Loss: 0.188362255692482
Epoch 391, Loss: 0.4190577566623688, Final Batch Loss: 0.20680972933769226
Epoch 392, Loss: 0.2920179218053818, Final Batch Loss: 0.14895601570606232
Epoch 393, Loss: 0.3912680

Epoch 503, Loss: 0.3246203362941742, Final Batch Loss: 0.16798636317253113
Epoch 504, Loss: 0.25662916898727417, Final Batch Loss: 0.10334569215774536
Epoch 505, Loss: 0.269232913851738, Final Batch Loss: 0.07675023376941681
Epoch 506, Loss: 0.3106527030467987, Final Batch Loss: 0.1423923671245575
Epoch 507, Loss: 0.23076261579990387, Final Batch Loss: 0.10051171481609344
Epoch 508, Loss: 0.35851408541202545, Final Batch Loss: 0.19241811335086823
Epoch 509, Loss: 0.24164288491010666, Final Batch Loss: 0.11194343119859695
Epoch 510, Loss: 0.23790276795625687, Final Batch Loss: 0.13593730330467224
Epoch 511, Loss: 0.28615058958530426, Final Batch Loss: 0.1218937337398529
Epoch 512, Loss: 0.29719504714012146, Final Batch Loss: 0.15060901641845703
Epoch 513, Loss: 0.24665574729442596, Final Batch Loss: 0.13634775578975677
Epoch 514, Loss: 0.3123062551021576, Final Batch Loss: 0.13279969990253448
Epoch 515, Loss: 0.34186458587646484, Final Batch Loss: 0.1649915724992752
Epoch 516, Loss: 0.2

...

Epoch 4486, Loss: 0.008333724457770586, Final Batch Loss: 0.0031619383953511715
Epoch 4487, Loss: 0.015500425361096859, Final Batch Loss: 0.00644429586827755
Epoch 4488, Loss: 0.016445515677332878, Final Batch Loss: 0.008565030992031097
Epoch 4489, Loss: 0.010623949347063899, Final Batch Loss: 0.002472369698807597
Epoch 4490, Loss: 0.012199576827697456, Final Batch Loss: 0.0016285422025248408
Epoch 4491, Loss: 0.015333617106080055, Final Batch Loss: 0.007928316481411457
Epoch 4492, Loss: 0.024313530884683132, Final Batch Loss: 0.008331202901899815
Epoch 4493, Loss: 0.051553234457969666, Final Batch Loss: 0.041500091552734375
Epoch 4494, Loss: 0.01286179618909955, Final Batch Loss: 0.0064186048693954945
Epoch 4495, Loss: 0.012399639468640089, Final Batch Loss: 0.0025559780187904835
Epoch 4496, Loss: 0.014058954315260053, Final Batch Loss: 0.0018956505227833986
Epoch 4497, Loss: 0.03393707424402237, Final Batch Loss: 0.006366649642586708
Epoch 4498, Loss: 0.02162258792668581, Final Batch

Epoch 4605, Loss: 0.030131075531244278, Final Batch Loss: 0.01573774218559265
Epoch 4606, Loss: 0.13035429641604424, Final Batch Loss: 0.1088944673538208
Epoch 4607, Loss: 0.016839356161653996, Final Batch Loss: 0.007954698987305164
Epoch 4608, Loss: 0.02300336118787527, Final Batch Loss: 0.012823645025491714
Epoch 4609, Loss: 0.02039624284952879, Final Batch Loss: 0.013466662727296352
Epoch 4610, Loss: 0.019939785357564688, Final Batch Loss: 0.013317347504198551
Epoch 4611, Loss: 0.01004459522664547, Final Batch Loss: 0.005695799365639687
Epoch 4612, Loss: 0.00822996674105525, Final Batch Loss: 0.004113702103495598
Epoch 4613, Loss: 0.01285489578731358, Final Batch Loss: 0.003114700550213456
Epoch 4614, Loss: 0.005878589232452214, Final Batch Loss: 0.0039500463753938675
Epoch 4615, Loss: 0.02304841112345457, Final Batch Loss: 0.01260290015488863
Epoch 4616, Loss: 0.01521255262196064, Final Batch Loss: 0.005932226777076721
Epoch 4617, Loss: 0.03866495564579964, Final Batch Loss: 0.0222

Epoch 4724, Loss: 0.00926003884524107, Final Batch Loss: 0.0021145115606486797
Epoch 4725, Loss: 0.022271830588579178, Final Batch Loss: 0.010880975052714348
Epoch 4726, Loss: 0.014961233828216791, Final Batch Loss: 0.0058098421432077885
Epoch 4727, Loss: 0.01457366906106472, Final Batch Loss: 0.011931739747524261
Epoch 4728, Loss: 0.023333203047513962, Final Batch Loss: 0.014307922683656216
Epoch 4729, Loss: 0.01162543031387031, Final Batch Loss: 0.0018365930300205946
Epoch 4730, Loss: 0.02502665831707418, Final Batch Loss: 0.0029797626193612814
Epoch 4731, Loss: 0.013108357321470976, Final Batch Loss: 0.004756383132189512
Epoch 4732, Loss: 0.02532186033204198, Final Batch Loss: 0.004027674440294504
Epoch 4733, Loss: 0.02167784608900547, Final Batch Loss: 0.004989545792341232
Epoch 4734, Loss: 0.01006748341023922, Final Batch Loss: 0.003110571764409542
Epoch 4735, Loss: 0.009711305378004909, Final Batch Loss: 0.002151738153770566
Epoch 4736, Loss: 0.007200982654467225, Final Batch Los

Epoch 4829, Loss: 0.032754594925791025, Final Batch Loss: 0.00634810933843255
Epoch 4830, Loss: 0.03000161936506629, Final Batch Loss: 0.024787675589323044
Epoch 4831, Loss: 0.009775524027645588, Final Batch Loss: 0.0026753144338726997
Epoch 4832, Loss: 0.015904540196061134, Final Batch Loss: 0.011646704748272896
Epoch 4833, Loss: 0.03407009597867727, Final Batch Loss: 0.02818324975669384
Epoch 4834, Loss: 0.0490814158692956, Final Batch Loss: 0.04189791902899742
Epoch 4835, Loss: 0.006810522871091962, Final Batch Loss: 0.0019774402026087046
Epoch 4836, Loss: 0.009872434195131063, Final Batch Loss: 0.005830601789057255
Epoch 4837, Loss: 0.021951002068817616, Final Batch Loss: 0.010725239291787148
Epoch 4838, Loss: 0.007693257764913142, Final Batch Loss: 0.001651021302677691
Epoch 4839, Loss: 0.05101561173796654, Final Batch Loss: 0.015923328697681427
Epoch 4840, Loss: 0.009080541785806417, Final Batch Loss: 0.0023400085046887398
Epoch 4841, Loss: 0.053442378994077444, Final Batch Loss:

Epoch 4948, Loss: 0.011447322554886341, Final Batch Loss: 0.002754436805844307
Epoch 4949, Loss: 0.014116973616182804, Final Batch Loss: 0.004097593016922474
Epoch 4950, Loss: 0.006144608370959759, Final Batch Loss: 0.002910233335569501
Epoch 4951, Loss: 0.008224709192290902, Final Batch Loss: 0.002956450218334794
Epoch 4952, Loss: 0.02235726686194539, Final Batch Loss: 0.017897235229611397
Epoch 4953, Loss: 0.015872520860284567, Final Batch Loss: 0.013526161201298237
Epoch 4954, Loss: 0.03556338371708989, Final Batch Loss: 0.006914852652698755
Epoch 4955, Loss: 0.014867953956127167, Final Batch Loss: 0.011757683008909225
Epoch 4956, Loss: 0.00663796765729785, Final Batch Loss: 0.00152506772428751
Epoch 4957, Loss: 0.021442533936351538, Final Batch Loss: 0.005959867965430021
Epoch 4958, Loss: 0.050656503066420555, Final Batch Loss: 0.04703480377793312
Epoch 4959, Loss: 0.006667634355835617, Final Batch Loss: 0.0017267422517761588
Epoch 4960, Loss: 0.02182060107588768, Final Batch Loss:

In [43]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_fake(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[16  2  0  0  0  0  4  0  0]
 [ 0  6  0  0  1  0  0  1  0]
 [ 0  0  7  0  0  1  0  0  0]
 [ 0  0  0  8  0  0  0  0  0]
 [ 0  0  0  0  8  0  0  0  0]
 [ 0  0  5  0  0  1  0  0  5]
 [ 1  4  0  0  0  0  6  0  1]
 [ 0  0  0  0  1  0  0  9  0]
 [ 0  0  8  0  0  0  0  0  5]]
              precision    recall  f1-score   support

           0    0.94118   0.72727   0.82051        22
           1    0.50000   0.75000   0.60000         8
           2    0.35000   0.87500   0.50000         8
           3    1.00000   1.00000   1.00000         8
           4    0.80000   1.00000   0.88889         8
           5    0.50000   0.09091   0.15385        11
           6    0.60000   0.50000   0.54545        12
           7    0.90000   0.90000   0.90000        10
           8    0.45455   0.38462   0.41667        13

    accuracy                        0.66000       100
   macro avg    0.67175   0.69198   0.64726       100
weighted avg    0.69515   0.66000   0.64617       100

